# 1. Import dependencies

In [2]:
#Modules en lien avec Gym
import gymnasium as gym

#Modules utiles
import sys
sys.path.append('/Users/mathiasperez/Documents/GitHub/PSC_Stratolia/Algo Naif')
from parcours import parcours_a_Z, distance_destination, case, ventU_ventV
import Node

import os
import numpy as np
import random
import os
from datetime import datetime, timedelta
import time

#Modules en lien avec Stable_baseline3
#IMPORT THE TYPE OF AGENT U WANT TO USE
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 1.bis Import the wind-datas

In [4]:
import pickle
with open("objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)

# 2. Create the environnement

In [ ]:
class Balloon(gym.Env):
    """
    ## Description
    This environment is a complete representation of our problem.

    The destination is always at variable coordinates (random opr not).
    So are the starting coordinatesv (probably)


    ## Action Space
    There are three discrete actions available:
    - 0: do nothing et let the wind lead your movement
    - 1: go up in altitude (17 heights allowed)
    - 2: go down in altitude (17 heights allowed)

    ## Observation Space
    The state a tuple consisting of different parts : 
        First you have the time accessible using the datetime module
        Then 2 Boxes (simple array) for the longitude and the latitude
        Finally one discrete part consisting of the 71 different heights

    ## Rewards
    After every step a reward is granted. The total reward of an episode is the
    sum of the rewards for all the steps within that episode.

    For each step, the reward:
    (- is increased/decreased the closer/further the ballon is to the goal point.)
    - is increased/decreased the slower/faster the lander is moving.
    (- is increased/decreased the less/more energy from the battery is used in the round)

    The episode receive an additional reward of +100 or -100 points for finding the goal or not respectively.



    ## Starting State
    The lander starts at a random poinf of the globe (time can be random as well)

    ## Episode Termination
    The episode finishes if:
    1) the balloon gets too far away from the goal;
    2) the balloon goes near enough of the goal

    ## Arguments

     * `goal_long` and 'goal_lat' for the position of our goal destination (can be fixed for specific results)
     * 'initial_time' defines the starting time of our episode (=> consequence on the winds)

    * `precision' defines how close the balloon must be to the target before being considered as having reached it.

    * 'limite_eloignement' defines how far the ballon must be from the target to consider the episode to be a failure.

    * `temps_exploration` defines how long we let the ballon follow the winds when action : 0 is considered

    ## Version History
    - v0: Initial version

    ## Current bugs

    ## Credits
    CMOI
    """
    def __init__(self):
        super(Balloon, self).__init__()

        self.goal_long = 2.65
        self.goal_lat = 34.54

        self.limite_eloignement = 10000 #en m
        self.precision = 1000 #en m

        self.initial_time = datetime(2021, 1, 1, 0, 0, 0)

        self.temps_exploration = 3600
        
        # Define the action space
        self.action_space = gym.spaces.Discrete(3)  
        """
        Action 0 : You do nothing and let the balloon follow the wind for temps_exploration seconds
        Action 1 : You push the balloon upwards (goes up in altitude) (instanneous action)
        Action 2 : You push the balloon downwards (goes down in altitude) (instanneous action)
        """
        
        # Define the observation space
        self.observation_space = gym.spaces.Tuple(( 
            gym.spaces.Box(low=datetime(2021, 1, 1, 0, 0, 0), high=datetime(2022, 1, 1, 0, 0, 0), shape=(1,), dtype=datetime), # time (in special format)
            gym.spaces.Discrete(18), #Altitude (17 different positions : no need for more precisions)
            gym.spaces.Box(low=0, high=180, shape=(1,), dtype=np.float32),  #Longitude
            gym.spaces.Box(low=0, high=90, shape=(1,), dtype=np.float32)   #Latitude
            ))
        
    def reset(self):
        # Reset the environment to its initial state
        # Resetting the ballon to its initial position : 73 boulevard des maréchaux
        #Ici on pourrait randomiser les goal et les positions de depart +temps
        self.state = (np.array([self.initial_time], dtype=datetime), 
                      0, 
                      np.array([2.23], dtype=np.float32),
                      np.array([50.23], dtype=np.float32)
                      )
        return self.state, {}
    
    def get_terminated(self, state):
        long, lat = state[2], state[3]
        if distance_destination((self.goal_long, self.goal_lat), long, lat) < self.limite_eloignement :
            return True
        return False
    
    def get_reward(self, state):
        """
        - fonction qui donne récompense en fonction de la distance  a l'objectif par rapport a la distance initale
        - ajoute un boolean si l'objectif a été atteint ou bien si on en est suffisamment proche
        - d'autres trucs a faire surement
        """
        pass

    
    def step(self, action):
        # Take a step in the environment based on the given action
        if action == 0:
            #You decide to follow the wind flow :
            long, lat, pression = self.state[2], self.state[3], self.state[1]
            secondes_depuis_debut = (self.state[0]-self.initial_time).total_seconds()
            temps = secondes_depuis_debut//(6*3600), secondes_depuis_debut%(6*3600)
            n = Node(long, lat,temps, pression, None) #longtiude, latitude, temps, ression, prev
            terminated, n = parcours_a_Z(
                                        (self.goal_long, self.goal_lat),
                                        n,
                                        temps_chgmt_pression = self.temps_exploration,
                                        precision = self.precision, 
                                        tab_vent = wind_data)
            done = terminated
            new_date = self.initial_time + timedelta(seconds=n.temps[0]*(6*3600)+n.temps[1]) # A corriger
            self.state = new_date, n.pression, n.long, n.lat

        elif action == 1:
            #You go up 1 case in altitude so u go down in pressure
            self.state[1] -= 1
        else :
            #You go down one case in altitudeso u go up in pressure
            self.state[1] += 1
    

        # Calculate the reward
        reward = get_reward(self.state)
        
        # Check if the episode has ended
        if not done:
            done = get_terminated(self.state)
        
        # Check if the episode has been truncated
        truncated = False
        
        # Return the observation, reward, done flag, truncated flag, and info
        return self.state, reward, done, truncated, {}
    
    def render(self, mode='human'):
        # Render the environment
        print("State:", self.state)


    
